# Curation of shebanq

By means of this notebook you can curate the shebanq website.

Curation consists of the following steps

1. Obtain a database dump of the `shebanq_web` and `shebanq_note` databases on web server of the
   shebanq.ancient-data.org website
2. Weed out all data that is not relevant to the published queries and shared notes, and remove all
   privacy-sensitive data
3. Check the Emdors databases of the hebrew data for gaps in the monad sequence
4. Create a mapping from the words of older versions to the words of the latest version, 2021
5. Create a TF set file containing the results of each published query as a separate set
6. Create html pages for each published query
7. Create an index page for the query pages

# Base directory

We assume your base directory for the github clones of the ETCBC repos is as follows.

`BASEDIR` is the directory under which you have your github organizations, such as ETCBC.
And under organizations you have your repos.
So if repo `ETCBC/shebanq-local` resides under `/your/specific/directory`, you should set

```
BASEDIR = "/your/specific/directory"
```

In [1]:
BASEDIR = "~/github"

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from helpers import SQL, Check, Mapper

# Latest databases with user contributed data

## No access to HuC infra

If you do not have access to the latest data, ask Dirk Roorda for backups.
You will get two files:

* `shebanq_web-all.sql.gz`
* `shebanq_web-all.note.gz`

Create a top-level directory `backup` in your `shebang-local` clone and put these backup files
in that directory.

## Otherwise, with access to HuC infra

If you do have access, you can work as follows, assuming you have access to the HuC infrastructure,
and you have the
[smartk8s](https://code.huc.knaw.nl/tt/smart-k8s)
suite installed.

Make a mysqldump of the real shebanq at ancient-data.org

```
k
kset shebanq
ksh
```

Now you are in a shell in the container where shebanq is being served on k8s.

```
cd src/scripts
./backup.sh ALL
exit
```

Now the mysqldumps are in the `/app/backup` directory, which is on a PVC.

Back in the shell on your own computer do

```
cd ~/github/ETCBC/shebanq-local
kfromapp backup
```

Now the backup directory has been copied from k8s to your local computer, in `app/backup`.

## In all cases

From this point onwards, we can work without any use of the HuC infra.

# Weed out the user data

From now on we work on the local computer, in the top-level directory of the clone
of the `shebanq-local` repo.

## Preparation

Start the local shebanq in production mode:

```
./shebanq.sh up
```

In another shell, still in directory `~/github/ETCBC/shebanq-local` do

```
./shebanq.sh sh
```

Now you are in a shell of the local shebanq, from where you can restore the backup we just
fetched from the production shebanq:

```
cd src/scripts
./restore.sh ALL
```

Now we export the data as sql definition files plus tsv data files, by means of mysqldump with the `--tab` option.

In the same shell where you did the restore, do

```
./export.sh
```

Now there are two directories added to the `app/backup` folder, namely `shebanq_web` and `shebanq_note`, and they
contain the definition and data files for each table in that database.

Open yet another shell in the same directory on your local computer.
Now you can explore the database by

```
./shebanq.sh browse admin
```

When asked for a password, it is `wajehior`.

## The weeding itself

Now we are in a position to weed out data from the exported files, after which we can re-import these files
by means of an import script.

If something goes wrong, we can always go back to the original backup by means of the restore script:

```
app sh
./restore.sh ALL
./export.sh
exit
```

We start by reading the tables into rows of fields in Python

In [6]:
S = SQL(BASEDIR, zapTables={"web2py_session_shebanq", "auth_event", "auth_cas"})
S.stats()

Database shebanq_note:
	Table note                     :   222527 rows
Database shebanq_web:
	Table auth_group               :     1675 rows
	Table auth_membership          :     1670 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1671 rows
	Table monads                   :  6295866 rows
	Table organization             :      529 rows
	Table project                  :      784 rows
	Table query                    :     7097 rows
	Table query_exe                :     8209 rows
	Table uploaders                :        1 rows


### Queries en notes

We remove all queries that are not published, and we remove all notes that are not shared.

Queries: weed out unpublished query executions:

In [7]:
S.keep("shebanq_web", "query_exe", lambda r: r[8] == "T")
S.stats()

Database shebanq_note:
	Table note                     :   222527 rows
Database shebanq_web:
	Table auth_group               :     1675 rows
	Table auth_membership          :     1670 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1671 rows
	Table monads                   :  6295866 rows
	Table organization             :      529 rows
	Table project                  :      784 rows
	Table query                    :     7097 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


Notes: weed out notes that are neither published nor shared:

In [8]:
S.keep("shebanq_note", "note", lambda r: r[9] == "T" or r[11] == "T")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1675 rows
	Table auth_membership          :     1670 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1671 rows
	Table monads                   :  6295866 rows
	Table organization             :      529 rows
	Table project                  :      784 rows
	Table query                    :     7097 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


Now we trim the database of all data that is related to the removed rows and is no longer needed.

We weed the `monads` detail records. 

In [9]:
S.trimDetails("shebanq_web", "query_exe", "shebanq_web", "monads", 0)
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1675 rows
	Table auth_membership          :     1670 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1671 rows
	Table monads                   :   701052 rows
	Table organization             :      529 rows
	Table project                  :      784 rows
	Table query                    :     7097 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


We weed the `query` master records.

In [10]:
S.trimMaster("shebanq_web", "query_exe", 10, "shebanq_web", "query")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1675 rows
	Table auth_membership          :     1670 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1671 rows
	Table monads                   :   701052 rows
	Table organization             :      529 rows
	Table project                  :      784 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


Remove the superfluous projects and organizations.

In [11]:
S.trimMaster("shebanq_web", "query", 9, "shebanq_web", "organization")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1675 rows
	Table auth_membership          :     1670 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1671 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      784 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


In [12]:
S.trimMaster("shebanq_web", "query", 8, "shebanq_web", "project")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1675 rows
	Table auth_membership          :     1670 rows
	Table auth_permission          :        0 rows
	Table auth_user                :     1671 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


We collect the users that are still connected to the current queries and notes.

In [13]:
userIds = S.getIds("shebanq_web", "query", 4) | S.getIds("shebanq_note", "note", 6) | S.getIds("shebanq_web", "uploaders", 0)

In [14]:
S.trimTable("shebanq_web", "auth_user", 0, userIds)
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1675 rows
	Table auth_membership          :     1670 rows
	Table auth_permission          :        0 rows
	Table auth_user                :      175 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


In [15]:
S.trimTable("shebanq_web", "auth_membership", 1, userIds)
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :     1675 rows
	Table auth_membership          :      175 rows
	Table auth_permission          :        0 rows
	Table auth_user                :      175 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


In [16]:
S.trimMaster("shebanq_web", "auth_membership", 1, "shebanq_web", "auth_group")
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :      175 rows
	Table auth_membership          :      175 rows
	Table auth_permission          :        0 rows
	Table auth_user                :      175 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


Strip data from remaining users.

In [17]:
S.zapFields("shebanq_web", "auth_user", 3, 4, 5, 6, 7)

A quick visual check on a the first 10 results:

In [18]:
S.data["shebanq_web"]["auth_user"][0:10]

[['2', 'Dirk', 'Roorda', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['4', 'Oliver', 'Glanz', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['11', 'Willem', 'van Peursen', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['13', 'Gino', 'Kalkman', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['15', 'Reinoud', 'Oosting', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['16', 'Martijn', 'Naaijer', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['19', 'Christiaan', 'Erwich', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['20', 'Joep', 'Dubbink', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['67', 'Femke', 'Siebesma', '\\N', '\\N', '\\N', '\\N', '\\N'],
 ['84', 'Nicolai', 'Winther-Nielsen', '\\N', '\\N', '\\N', '\\N', '\\N']]

Write the weeded data back to the files:

In [19]:
S.writeData()

## Re-import

When the weeding is done, import the new data as follows:

```
app sh
./import.sh
```

## Backup

If all is well, make a backup dump and put it in the content folder.

still in the same shell as above:

```
./backup.sh
exit
```

And then in the local shell:

```
mv backup/shebanq_web.sql.gz backup/shebanq_note.sql.gz content
```


# Check the Emdros databases

This step is just a sanity check, it will not influence subsequent processing.

In Emdros databases, the sequences of monads do not have to be consecutive.
But when we prepared shebanq, we have made those sequences consecutive.

Here is a check to see whether this requirement still holds.
We take the emdros databases from a clone of ETCBC/bhsa, assuming this clone sits next to shebanq-local on your computer.

We need to unzip the bzipped data files. 
We unzip them to a temporary folder, if they are not already there.

In [23]:
C = Check(BASEDIR)

In [27]:
C.unzip()

unzipping ~/github/ETCBC/bhsa/shebanq/4/shebanq_etcbc4.mql.bz2
unzipping ~/github/ETCBC/bhsa/shebanq/4b/shebanq_etcbc4b.mql.bz2
unzipping ~/github/ETCBC/bhsa/shebanq/2016/shebanq_etcbc2016.mql.bz2
unzipping ~/github/ETCBC/bhsa/shebanq/2017/shebanq_etcbc2017.mql.bz2
unzipping ~/github/ETCBC/bhsa/shebanq/c/shebanq_etcbcc.mql.bz2
unzipping ~/github/ETCBC/bhsa/shebanq/2021/shebanq_etcbc2021.mql.bz2


In [28]:
C.monads()

Checking 4 from ~/github/ETCBC/shebanq-local/_temp/shebanq_etcbc4.mql
	last monad = 426555
	there were 0 gaps
Checking 4b from ~/github/ETCBC/shebanq-local/_temp/shebanq_etcbc4b.mql
	last monad = 426568
	there were 0 gaps
Checking 2016 from ~/github/ETCBC/shebanq-local/_temp/shebanq_etcbc2016.mql
	last monad = 426581
	there were 0 gaps
Checking 2017 from ~/github/ETCBC/shebanq-local/_temp/shebanq_etcbc2017.mql
	last monad = 426584
	there were 0 gaps
Checking c from ~/github/ETCBC/shebanq-local/_temp/shebanq_etcbcc.mql
	last monad = 426584
	there were 0 gaps
Checking 2021 from ~/github/ETCBC/shebanq-local/_temp/shebanq_etcbc2021.mql
	last monad = 426590
	there were 0 gaps


# Map slots of all non-2021 versions to slots of 2021

We want to show all query results in Text-Fabric, using only one version of the BHSA, the 2021 version.

To that end, we have to map the slots (word positions) of all other versions to those of the 2021 version.
Every version of the BHSA has a feature that contains the mapping from older version slots to newer version slots.

We load all versions in Text-Fabric:

In [42]:
M = Mapper(BASEDIR)

During the execution of the following cell you'll see some errors in the 4, 4b versions having to do with the feature `voc_lex_utf8`.
This is not a problem.

In [43]:
M.load()

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10937.31,100
chapter,929,459.16,100
lex,9225,46.24,100
verse,23213,18.38,100
half_verse,44682,9.55,100
sentence,66045,6.46,100
sentence_atom,66701,6.40,100
clause,87978,4.85,100
clause_atom,90144,4.73,100
phrase,254664,1.67,100


App config error(s) in lex:
	template: feature voc_lex_utf8 not loaded
	label: feature voc_lex_utf8 not loaded


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10937.64,100
chapter,929,459.17,100
lex,9238,46.18,100
verse,23213,18.38,100
half_verse,45180,9.44,100
sentence,63586,6.71,100
sentence_atom,64354,6.63,100
clause,88011,4.85,100
clause_atom,90554,4.71,100
phrase,253161,1.68,100


...

Name,# of nodes,# slots / node,% coverage
book,39,10938.05,100
chapter,929,459.19,100
lex,9233,46.20,100
verse,23213,18.38,100
half_verse,45180,9.44,100
sentence,63711,6.70,100
sentence_atom,64486,6.62,100
clause,88101,4.84,100
clause_atom,90669,4.70,100
phrase,253187,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.05,100
chapter,929,459.19,100
lex,9233,46.20,100
verse,23213,18.38,100
half_verse,45180,9.44,100
sentence,63727,6.69,100
sentence_atom,64525,6.61,100
clause,88121,4.84,100
clause_atom,90688,4.70,100
phrase,253207,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


Now we compute the mappings between older versions and the 2021 version.
While we do that, we check if there are unmapped slots.

In [44]:
M.makeMappings()

map c-slots to 2021 slots ...
	0 gaps
map 2017-slots to 2021 slots ...
	0 gaps
map 2016-slots to 2021 slots ...
	0 gaps
map 4b-slots to 2021 slots ...
	0 gaps
map 4-slots to 2021 slots ...
	0 gaps


We do an extra check on gaps:

In [45]:
M.checkGaps()

mapping 4 to 2021 has 0 gaps
mapping 4b to 2021 has 0 gaps
mapping 2016 to 2021 has 0 gaps
mapping 2017 to 2021 has 0 gaps
mapping c to 2021 has 0 gaps


Because the loaded TF datasets take a lot of memory, we are going to unload them:

# Some values

In [46]:
M.showValues(100000, 200000, 300000, 400000)

   c-slot 100000 maps to 100001
2017-slot 100000 maps to 100001
2016-slot 100000 maps to 100002
  4b-slot 100000 maps to 100008
   4-slot 100000 maps to 100010

   c-slot 200000 maps to 200003
2017-slot 200000 maps to 200003
2016-slot 200000 maps to 200004
  4b-slot 200000 maps to 200014
   4-slot 200000 maps to 200021

   c-slot 300000 maps to 300003
2017-slot 300000 maps to 300003
2016-slot 300000 maps to 300004
  4b-slot 300000 maps to 300014
   4-slot 300000 maps to 300022

   c-slot 400000 maps to 400004
2017-slot 400000 maps to 400004
2016-slot 400000 maps to 400007
  4b-slot 400000 maps to 400016
   4-slot 400000 maps to 400027



# Turn the query results into a TF set

We save the query result of all published queries as a TF set in the contents directory.

In [49]:
S.writeQResultsTF(M.mappingsFrom)

# Write documentation pages for the queries

We start with loading the object with the query information, in case we are starting here.

In [4]:
S = SQL(BASEDIR, zapTables={"web2py_session_shebanq", "auth_event", "auth_cas"})
S.stats()

Database shebanq_note:
	Table note                     :   222517 rows
Database shebanq_web:
	Table auth_group               :      175 rows
	Table auth_membership          :      175 rows
	Table auth_permission          :        0 rows
	Table auth_user                :      175 rows
	Table monads                   :   701052 rows
	Table organization             :       75 rows
	Table project                  :      112 rows
	Table query                    :     1130 rows
	Table query_exe                :     1157 rows
	Table uploaders                :        1 rows


In [13]:
S.genQueryPages()

Cleaning previous results ... 
Gathering projects ... 
Gathering organizations ... 
Gathering users ... 
Gathering queries ... 
Gathering query executions ... 
Generating pages ... 
Generated 1157 pages for 1130 queries
